In [1]:
from datetime import datetime

import pandas as pd
import numpy as np

import codecs, json 

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from sklearn.model_selection import train_test_split

import xgboost as xgb

from sklearn.metrics import mean_squared_error

import scipy

In [2]:
######################### METHODS ###################################

In [3]:
def parse_data(filename):
    with open(filename) as json_file:
        json_data = json.load(json_file)
    
    frames = []
    for j in range(len(json_data['tables'])):
        df = pd.DataFrame(np.array(json_data['tables'][j]['table'])[:,:], 
                           columns = json_data['tables'][j]['header']['variables'][:])
        df['state'] = json_data['tables'][j]['header']['flightphase']
        if df['state'][0] == 'cruise':
            frames.append(df)
    
    df = pd.concat(frames,ignore_index=True)
    return df

In [4]:
def getInterpolation(temp, alt, mass, speed, objective, filename):
    df = parse_data(filename)
    df = df[['DISA','ALTITUDE','MASS','MACH', objective]]
    
    test = pd.concat([temp, alt, mass, speed, objective])
    df.append(test) # where to place points? or does not matter... (see previous tests)
    
    return ((df.interpolate()).iloc[test.index.values])[objective]
    

In [5]:
####################################################################

In [6]:
with open('data_rich_ac.bsad') as json_file:
    json_data = json.load(json_file)

In [7]:
# json_data is a disc 
# The tables is a list and header is dict
# The tables list contains dict -> This dict contains the data
#json_data['tables'][1]['table']


In [213]:
frames = []
for j in range(len(json_data['tables'])):
    df = pd.DataFrame(np.array(json_data['tables'][j]['table'])[:,:], 
                           columns = json_data['tables'][j]['header']['variables'][:])
    df['state'] = json_data['tables'][j]['header']['flightphase']
    if df['state'][0] == 'cruise':
        frames.append(df)
    
df = pd.concat(frames,ignore_index=True)
df = df[['DISA','ALTITUDE','MASS','MACH','FUELFLOW']]

In [279]:
X = df.drop(['FUELFLOW'], axis=1)
y = df.FUELFLOW

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

#X_train.insert(4, 'FUELFLOW', y_train, False)
#X = X_train.append(X_test)
#X = X.sort_index(axis=0)
test = X_test.iloc[0]
y_check = y_test.iloc[0]


In [280]:
X_train['distance'] = X_train.sub(test).pow(2).sum(1).pow(0.5) # maybe cutoff based on input
z = X_train.sort_values('distance').iloc[0:4]
z = z.drop(['distance'],axis = 1)
z = z.reset_index()
z.columns = ['index_', 'DISA', 'ALTITUDE', 'MASS', 'MACH']
y_train.columns = ['index_', 'FUELFLOW']
z = z.join(y_train, on = 'index_', how = 'left')
#z = z.drop(['index_'], axis=1)

/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,index_,DISA,ALTITUDE,MASS,MACH,FUELFLOW
0,29069,-20.0,0.0,64000.0,0.30,0.274195
1,30138,-20.0,0.0,64000.0,0.32,0.277547
2,27130,-20.0,0.0,64000.0,0.24,0.296419
3,31341,-20.0,0.0,64000.0,0.34,0.284537


In [289]:
z_inter = z.append(test)

z_inter = z[0:2].append(test)
z_inter = z_inter.append(z[2:4])
y_inter = z_inter.interpolate(method = 'linear')['FUELFLOW']
y_inter.iloc[2] - y_check
#mean_squared_error(y_inter.iloc[2], y_check)

0.010332000000000008

In [206]:
#X_train.sort_values('distance').iloc[0:5]
#X_train.index.values
#y_train.iloc[]

test

DISA          -30.0
ALTITUDE    27000.0
MASS        44000.0
MACH            0.7
Name: 70905, dtype: float64